In [ ]:
#import relevant libraries
import tensorflow as tf
import pandas as pd
import numpy as np

In [ ]:
# Load data
data = pd.read_csv('../data/c8y_Acceleration_Merged_Shuffle.csv')
data.sample(5)

In [ ]:
#remove time column from the source
data.pop(data.columns[0])
data.sample(5)

In [ ]:
# pop the label columns
labels = data.pop("label")
print(labels)


In [ ]:
# normalize data
data = (data - data.mean()) / data.std() 
data.sample(5)

In [ ]:
# Reshape data to 3D array
data = data.values.reshape((-1, 3, 1))
print(data)

In [ ]:
# Convert labels to binary
labels = np.where(labels=="moving", 1, 0)

In [ ]:
# Split data into train and test sets
split = int(len(data) * 0.8)
x_train = data[:split]
y_train = labels[:split]
x_test = data[split:]
y_test = labels[split:]

In [ ]:
# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(3, 1)),
    tf.keras.layers.Conv1D(32, 3, activation='relu', padding='same'),
    tf.keras.layers.Reshape((3, 32, 1)),
    tf.keras.layers.MaxPooling2D((2, 1)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=30, batch_size=64)

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)

In [ ]:
# Use the model to predict label for a single row of data
single_row = np.array([[[0.070770263671875,0.7673492431640625,0.751739501953125]]]).reshape((1, 3, 1))
#single_row = np.array([1.0,1.0,1.0]).reshape((1, 3, 1))
prediction = model.predict(single_row)
prediction 

# Save Model as Protocol Buffer Format

In [ ]:
#MODEL_DIR = tempfile.gettempdir()
version = 1
#export_path = os.path.join(MODEL_DIR, str(version))
#print('export_path = {}\n'.format(export_path))

export_path = '../model/cumulocity_classification_model_3/0000001'

# Export the model as a SavedModel
tf.saved_model.save(model, export_path)


print('\nSaved model:')
#!ls -l {export_path}

# Convert TensorFlow to ONNX

#Single Line Command: https://onnxruntime.ai/docs/tutorials/tf-get-started.html